# Análisis univariado de independencia entre variables a partir del test Chi-cuadrado

En este ejemplo, se busca determinar si es que existe dependencia entre una variable de interes y posibles candidatos que puedan explicar su comportamiento. Para esto emplearemos la prueba de chi-cuadrado, la cual compara los valores observados con los esperados, y cuya hipotesis nula planetea que no existe dependencia.

>${H_0}:$ La variable de interes Y es independiente de la variable X

>${H_1}:$ La variable de interes Y es diferente dependiendo de la variable X

Para este ejemplo en particular generaremos data sintetica de una variable de interes "y" (dependiente), junto con 3 variables independientes que puedan guardar relación con ella (de antemano sabemos que las 2 primeras si lo hacen y la tercera no, ya que así lo definimos en la generacion de información).

La forma de realizar la prueba de hipotesis es a través del estadistico chi cuadrado, el cual se calcula mediante la siguiente expresión:

$${X^2}=\sum\frac{(O-E)^2}{E}\sim{X^2_{(r-1)(c-1)}}$$

Donde:

$O_i$: Representa a cada frecuencia observada

$E_i$: Representa a cada frecuencia esperada

$(r-1)(c-1)$: Representa los grados de libertad

Una vez calculado el estadistico, es posible obtener el valor P asociado a este. Para esto, es necesario emplear la distribucion chi cuadrado acorde a los grados de libertad del ejercicio, estos vienen dados por el numero de columnas y filas de la matriz de frecuencias (frecuencia de las combinaciones de categorias pertenecientes a ambas variables) menos 1. Si el valor P resultante es inferior a la significancia (α, generalmente 0.05) rechazamos la hipotesis nula dada la baja probabilidad de obtener ese resultado debido meramente al azar.

In [66]:
import numpy as np
from scipy import stats
from numpy.random import default_rng
import pandas as pd 

# Generamos las 3 variables independientes, para explorar su relacion con la variable dependiente
X = default_rng(123).integers(0,3,size=(3000,3)) 
n, p = 2, 0.5

# Generamos la variable dependiente en funcion de las primeras 2 variables, el ultimo termino es ruido en dist binomial
y = X[:,0]//2 + X[:,1] + stats.binom(n,p).rvs(size=3000, random_state=123)


# Para el correcto desempeño del estadistico se requiere al menos 1 valor de frecuencia observada por combinación.
# por lo que agregamos valores que quedan fuera del rango de formula generada para y.
extra = np.array([[2,1,0,0]
                 ,[0,0,2,5]
                 ,[1,1,0,5]
                 ,[1,0,2,4]
                 ,[1,2,2,1]
                 ,[1,2,2,0]])

# Consolidamos la informacion en un Dataframe
df = pd.DataFrame(np.concatenate([np.c_[X,y],extra], axis=0),columns=('x1','x2','x3','y'))

Construimos nuestras tablas de contingencia para los valores observados de acuerdo a los 3 posibles valores de cada variable, en funcion de los valores de la variable dependiente "y":

In [67]:
piv_x1 = pd.pivot_table(data=df, index='x1',columns='y',values='x2',aggfunc='count',margins=True)
piv_x2 = pd.pivot_table(data=df, index='x2',columns='y',values='x1',aggfunc='count',margins=True)
piv_x3 = pd.pivot_table(data=df, index='x3',columns='y',values='x1',aggfunc='count',margins=True)
piv_x3

y,0,1,2,3,4,5,All
x3,,,,,,,
0,66,186,315,275,134,22,998
1,57,204,304,289,138,14,1006
2,59,206,309,255,147,26,1002
All,182,596,928,819,419,62,3006


A continuacion, construiremos una función que nos permita obtener la tabla de contingencia de valores esperados asumiendo la hipotesis nula como verdadera, es decir, que sean variables independientes entre si:

In [68]:
def expected_freq_tab(df_table):
    table = df_table
    size = df_table.iloc[-1,-1]
    col = 6
    row = 3
    l=[]
    for i in range(col):
        for j in range(row):
            l.append(df_table[i]['All']/size*df_table['All'][j]/size*size)

    return pd.DataFrame(np.array(l).reshape(6,3).T, columns=list(range(6)))  # Matriz de valores esperados

Dado que ya contamos con ambas tablas de contingencia, para valores observados y esperados, podemos calcular el estadistico chi cuadrado para cada variable en funcion de y:

In [69]:
def p_value(df_table, grad_lib):
    chi = (df_table.iloc[0:3,0:6] - expected_freq_tab(df_table))**2/expected_freq_tab(df_table) # Calculo del estadistico
    chi_stat = chi.sum().sum()
    return stats.chi2.sf(chi.sum().sum(),grad_lib)  # valor P para el estadistico

In [70]:
grad_lib = (3-1)*(6-1)
print('Definiendo una significancia de 0.05, se rechaza H0 si el valor p es inferior a este, por lo que:')
print(f' El valor p para x1 e y es :{p_value(piv_x1, grad_lib):.1e} por ende hay suficiente evidencia estadistica para rechazar H0, no son independientes')
print(f' El valor p para x2 e y es :{p_value(piv_x2, grad_lib):.1e} por ende hay suficiente evidencia estadistica para rechazar H0, no son independientes')
print(f' El valor p para x3 e y es :{p_value(piv_x3, grad_lib):.1f} por ende NO hay suficiente evidencia estadistica para rechazar H0, y asumo la hipotesis nula como verdadera, es decir, son independientes')

Definiendo una significancia de 0.05, se rechaza H0 si el valor p es inferior a este, por lo que:
 El valor p para x1 e y es :3.4e-89 por ende hay suficiente evidencia estadistica para rechazar H0, no son independientes
 El valor p para x2 e y es :0.0e+00 por ende hay suficiente evidencia estadistica para rechazar H0, no son independientes
 El valor p para x3 e y es :0.6 por ende NO hay suficiente evidencia estadistica para rechazar H0, y asumo la hipotesis nula como verdadera, es decir, son independientes


**Conclusion:** Contamos con evidencia suficiente evidencia estadistica para afirmar que existe una asociacion entre la variable dependiente "y" y las variables "x1" y "x2", mientras que no podemos concluir esto para "x3", todo esto con un nivel de significancia de un 5%